#📌 Session 1 : Import des modules & définition des variables

In [1]:

import imaplib
import email
import re
import os
import datetime
from email.header import decode_header
import dropbox
EMAIL = "habitategmbh@gmail.com"
PASSWORD = "p e e t l b d k i q w z a g y l"  # PAS ton mot de passe Gmail normal



#📌 Session 2 : Connexion IMAP 

In [2]:
try:
    imap = imaplib.IMAP4_SSL("imap.gmail.com")
    imap.login(EMAIL, PASSWORD)
    imap.select("inbox")
    print("✅ Connexion IMAP réussie")
except Exception as e:
    print(f"❌ Échec de connexion IMAP : {e}")


✅ Connexion IMAP réussie


📌# Session 3 : Connexion Dropbox

In [3]:
import dropbox
APP_KEY = 'ahpwm4u91q58ovc'
APP_SECRET = 'szvfv6e0ncxnxjs'
REFRESH_TOKEN = "twfO_afMFdQAAAAAAAAAAaL0dnKnaHyA8x79McRLrvnidqFvEwbc7dianZ-XSKCQ"

# Connexion avec refresh token
try:
 dbx = dropbox.Dropbox(
    oauth2_refresh_token=REFRESH_TOKEN,
    app_key=APP_KEY,
    app_secret=APP_SECRET
 )
 print("✅ Connexion à Dropbox réussie")
except Exception as e:
 print(f"❌ Erreur de connexion à Dropbox : {e}")

✅ Connexion à Dropbox réussie


#📌 Session 4 : Recherche des emails de 2025

In [4]:
try:
    status, messages = imap.search(None, 'SINCE', '"01-Jan-2025"', 'BEFORE', '"01-Jan-2026"')
    mail_ids = messages[0].split()
    print(f"✅ {len(mail_ids)} email(s) trouvé(s) pour l’année 2025")
except Exception as e:
    print(f"❌ Erreur lors de la recherche des mails : {e}")


✅ 1218 email(s) trouvé(s) pour l’année 2025


#📌 Session 5 : Filtrage et envoi dans Dropbox

In [5]:
import time  # si ce n'est pas encore fait

for num in mail_ids:
    try:
        try:
            status, data = imap.fetch(num, "(RFC822)")
        except Exception as e:
            print(f"🔁 Erreur de connexion : {e} → Reconnexion en cours...")
            try:
                imap = imaplib.IMAP4_SSL("imap.gmail.com")
                imap.login(EMAIL, PASSWORD)
                imap.select("inbox")
                status, data = imap.fetch(num, "(RFC822)")
                print("✅ Reconnexion réussie")
            except Exception as reconnection_error:
                print(f"❌ Reconnexion échouée : {reconnection_error}")
                continue  # saute ce mail

        msg = email.message_from_bytes(data[0][1])

        subject, encoding = decode_header(msg["Subject"])[0]
        if isinstance(subject, bytes):
            subject = subject.decode(encoding or "utf-8", errors="ignore")

        # Vérifier si l’e-mail est bien de l’année 2025
        date_tuple = email.utils.parsedate_tz(msg["Date"])
        date_obj = datetime.datetime.fromtimestamp(email.utils.mktime_tz(date_tuple))

        if date_obj.year == 2025 and "Lieferung" in subject:
            formatted_date = date_obj.strftime("%Y-%m-%d_%H%M")

            # Parcourir les parties du message
            for part in msg.walk():
                content_disposition = part.get("Content-Disposition", "")
                if "attachment" in content_disposition:
                    filename = part.get_filename()
                    if filename:
                        # Décoder le nom si besoin
                        filename, encoding = decode_header(filename)[0]
                        if isinstance(filename, bytes):
                            filename = filename.decode(encoding or "utf-8", errors="ignore")

                        if filename.lower().endswith(".pdf"):
                            local_name = f"{formatted_date}_{re.sub(r'[^\w]+', '_', subject)}_{filename}"
                            dropbox_path = f"/Dossier1/Lieferung_2025/{local_name}"

                            # Sauvegarder temporairement le fichier
                            with open("temp_attachment.pdf", "wb") as f:
                                f.write(part.get_payload(decode=True))

                            # Upload dans Dropbox
                            with open("temp_attachment.pdf", "rb") as f:
                                dbx.files_upload(f.read(), dropbox_path, mode=dropbox.files.WriteMode.overwrite)

                            print(f"✅ PDF '{filename}' sauvegardé dans Dropbox")

        time.sleep(1)  # pause pour éviter blocage

    except Exception as e:
        print(f"❌ Échec pour un mail : {e}")


✅ PDF 'Lieferung 0801790239.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801790346.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801790832.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801791253.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801791260.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801791436.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801791444.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801791906.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801793046.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801793046.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801793384.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801793454.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801794146.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801794450.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801794717.PDF' sauvegardé dans Dropbox
✅ PDF 'Würth_Lieferschein_8103778812.pdf' sauvegardé dans Dropbox
✅ PDF 'Lieferung 0801795281.PDF' sauvegardé dans Dropbox
✅ PDF 'Lieferung 08017

In [ ]:
import time  # si ce n'est pas encore fait

for num in mail_ids:
    try:
        try:
            status, data = imap.fetch(num, "(RFC822)")
        except Exception as e:
            print(f"🔁 Erreur de connexion : {e} → Reconnexion en cours...")
            try:
                imap = imaplib.IMAP4_SSL("imap.gmail.com")
                imap.login(EMAIL, PASSWORD)
                imap.select("inbox")
                status, data = imap.fetch(num, "(RFC822)")
                print("✅ Reconnexion réussie")
            except Exception as reconnection_error:
                print(f"❌ Reconnexion échouée : {reconnection_error}")
                continue  # saute ce mail

        msg = email.message_from_bytes(data[0][1])

        subject, encoding = decode_header(msg["Subject"])[0]
        if isinstance(subject, bytes):
            subject = subject.decode(encoding or "utf-8", errors="ignore")

        # Vérifier si l’e-mail est bien de l’année 2025
        date_tuple = email.utils.parsedate_tz(msg["Date"])
        date_obj = datetime.datetime.fromtimestamp(email.utils.mktime_tz(date_tuple))

        if date_obj.year == 2025 and "Rechnung" in subject:
            formatted_date = date_obj.strftime("%Y-%m-%d_%H%M")

            # Parcourir les parties du message
            for part in msg.walk():
                content_disposition = part.get("Content-Disposition", "")
                if "attachment" in content_disposition:
                    filename = part.get_filename()
                    if filename:
                        # Décoder le nom si besoin
                        filename, encoding = decode_header(filename)[0]
                        if isinstance(filename, bytes):
                            filename = filename.decode(encoding or "utf-8", errors="ignore")

                        if filename.lower().endswith(".pdf"):
                            local_name = f"{formatted_date}_{re.sub(r'[^\w]+', '_', subject)}_{filename}"
                            dropbox_path = f"/Dossier1/Rechnung_2025/{local_name}"

                            # Sauvegarder temporairement le fichier
                            with open("temp_attachment.pdf", "wb") as f:
                                f.write(part.get_payload(decode=True))

                            # Upload dans Dropbox
                            with open("temp_attachment.pdf", "rb") as f:
                                dbx.files_upload(f.read(), dropbox_path, mode=dropbox.files.WriteMode.overwrite)

                            print(f"✅ PDF '{filename}' sauvegardé dans Dropbox")

        time.sleep(1)  # pause pour éviter blocage

    except Exception as e:
        print(f"❌ Échec pour un mail : {e}")


🔁 Erreur de connexion : command: FETCH => unexpected response: b'MDAwMDwveG1wRzpibGFjaz4KICAgICAgICAgICAgICAg' → Reconnexion en cours...
✅ Reconnexion réussie
✅ PDF 'Rechnung 0091781407.PDF' sauvegardé dans Dropbox
